In [ ]:

# This module  is used to create a Google API service object for interacting with various Google services.
from googleapiclient.discovery import build

# This module provides various functions to parse and manipulate ISO 8601 date/time strings.
import isodate

# This module  represents a duration, which is the difference between two dates or times.
from datetime import timedelta


Google Developers Console - https://console.developers.google.com/ # It provides a central place to create and manage projects, enable APIs, set up credentials (like API keys, OAuth client IDs, service accounts, etc.), monitor usage, and access billing information. 
Google API Python Client - https://github.com/googleapis/google-api-python-client #t acts as a Python wrapper for Google APIs, providing a convenient and consistent interface to interact with these APIs.
YouTube API - https://developers.google.com/youtube/v3 #It provides methods to search for videos, retrieve video information, manage playlists, get channel data, and perform other YouTube-related tasks. 


In [ ]:

# Building a YouTube API client using the developer key provided (you should replace " " with your actual YouTube Data API developer key).
youtube = build("youtube", "v3", developerKey="YOUR_YOUTUBE_DATA_API_KEY")

# This query fetches the statistics of a channel with the specified 'forUsername'.
#ForUSername = Youtube username for user
channel_id = youtube.channels().list(part="statistics", forUsername=' ')

# This query fetches the playlists with both 'contentDetails' and 'snippet' parts for the specified 'channelId'.
playlist = youtube.playlists().list(part="contentDetails, snippet", channelId=' ')

Once the above code is executed once, we only need to french any playlistId and insert the playitem

In [ ]:
# Initializing 'nextPageToken' to None for the while loop
nextPageToken = None

while True:
    # Retrieving playlist items with both 'contentDetails' and 'snippet' parts for the specified 'playlistId'
    # Also, setting 'maxResults' to 50 to get a maximum of 50 items per request and using 'pageToken' to handle pagination.
    playitem = youtube.playlistItems().list(
        part="contentDetails,snippet",
        playlistId='',
        maxResults=50,
        pageToken=nextPageToken
    )#insert your playlistId by going to any youtube channel, click on specified playlist folder and check the url
    # Executing the API request
    pl_response = playitem.execute()

    # Creating an empty list to store the video IDs from the playlist response
    Video_id = []
    for item in pl_response["items"]:
        # Extracting video IDs from the playlist response and appending them to the 'Video_id' list
        ids = item['contentDetails']['videoId']
        Video_id.append(ids)
    
    # Retrieving video details for the extracted video IDs using the 'videos().list()' method
    # This query fetches video details with the 'contentDetails' part for the specified video IDs.
    video_request = youtube.videos().list(
        part='contentDetails',
        id=",".join(Video_id)
    )
    # Executing the API request
    video_response = video_request.execute()

    # Calculating the total duration of all videos in seconds
    total_seconds = 0
    for item in video_response["items"]:
        # Extracting video durations from the video response and parsing them using 'isodate' module
        ids = item['contentDetails']['duration']
        dt = isodate.parse_duration(ids)
        total_seconds += dt.total_seconds()
    total_seconds = int(total_seconds)

    # Converting the total duration to hours, minutes, and seconds
    minutes, seconds = divmod(total_seconds, 60)
    hours, minutes = divmod(minutes, 60)

    # Updating the 'nextPageToken' for the next page of playlist items, if available
    nextPageToken = pl_response.get("nextPageToken")
    if not nextPageToken:
        break  # Break the loop if there are no more pages

# Printing the total duration of all videos in the playlist
print(f'Total video time: {hours}H:{minutes}M:{seconds}S')

